<img src="data/photutils_banner.svg" width=500 alt="Photutils logo">

# Photutils

- Code: https://github.com/astropy/photutils
- Documentation: http://photutils.readthedocs.org/en/stable/
- Issue Tracker:  https://github.com/astropy/photutils/issues

## Photutils can be used for:

- Background and background noise estimation
- Source Detection and Extraction
  - DAOFIND and IRAF's starfind
  - Image segmentation
  - local peak finder
- Aperture photometry
- PSF photometry
- PSF matching
- Centroids
- Morphological properties
- Elliptical isophote analysis


## In this section, we will:

- Learn how to perform aperture photometry
- Learn how to use photutils' image segmentation module

---

## Preliminaries

In [ ]:
# Initial imports
import numpy as np
import matplotlib.pyplot as plt

# Change some default plotting parameters
import matplotlib as mpl
mpl.rcParams['image.origin'] = 'lower'
mpl.rcParams['image.interpolation'] = 'nearest'
from matplotlib.colors import LogNorm

# Run the %matplotlib magic command to enable inline plotting
# in the current notebook.  Choose one of these:
%matplotlib inline
# %matplotlib notebook

### Load the data

We'll start by reading data and error arrays from FITS files.  These are cutouts from the HST Extreme-Deep Field (XDF) taken with WFC3/IR in the F160W filter.

In [ ]:
from astropy.io import fits

In [ ]:
sci_fn = 'data/xdf_hst_wfc3ir_60mas_f160w_sci.fits'
rms_fn = 'data/xdf_hst_wfc3ir_60mas_f160w_rms.fits'
sci_hdulist = fits.open(sci_fn)
rms_hdulist = fits.open(rms_fn)

sci_hdulist[0].header['BUNIT'] = 'electron/s'

Print some info about the data.

In [ ]:
sci_hdulist.info()

Define the data and error arrays.

In [ ]:
data = sci_hdulist[0].data.astype(np.float)
error = rms_hdulist[0].data.astype(np.float)

Extract the data header and create a WCS object.

In [ ]:
from astropy.wcs import WCS

In [ ]:
hdr = sci_hdulist[0].header
wcs = WCS(hdr)

Display the data.

In [ ]:
from astropy.visualization import ImageNormalize, LogStretch

In [ ]:
norm = ImageNormalize(vmin=1e-4, vmax=5e-2, stretch=LogStretch())
plt.imshow(data, norm=norm)
plt.title('XDF F160W Cutout')

---
# Part 1:  Aperture Photometry

Photutils provides circular, elliptical, and rectangular aperture shapes (plus annulus versions of each).  These are names of the aperture classes, defined in pixel coordinates:

* `CircularAperture`
* `CircularAnnulus`

* `EllipticalAperture`
* `EllipticalAnnulus`

* `RectangularAperture`
* `RectangularAnnulus`

Along with variants of each, defined in celestial coordinates:

* `SkyCircularAperture`
* `SkyCircularAnnulus`

* `SkyEllipticalAperture`
* `SkyEllipticalAnnulus`

* `SkyRectangularAperture`
* `SkyRectangularAnnulus`

These look something like this:
<img src='data/apertures.png' alt='Figure of aperture shapes' width=700px>

## Methods for handling aperture/pixel intersection

In general, the apertures will only partially overlap some of the pixels in the data.

There are three methods for handling the aperture overlap with the pixel grid of the data array.

<img src="data/photutils_aperture_methods.svg">

NOTE:  the `subpixels` keyword is ignored for the **'exact'** and **'center'** methods.

### Perform circular-aperture photometry on some sources in the XDF

First, we define a circular aperture at a given position and radius (in pixels).

In [ ]:
from photutils import CircularAperture

In [ ]:
position = (90.73, 59.43)  # (x, y) pixel position
radius = 5.  # pixels
aperture = CircularAperture(position, r=radius)

In [ ]:
aperture

In [ ]:
print(aperture)

We can plot the aperture on the data using the aperture `plot()` method:

In [ ]:
plt.imshow(data, norm=norm)
aperture.plot(color='red', lw=2)

Now let's perform photometry on the data using the `aperture_photometry()` function.  **The default aperture method is 'exact'.**

Also note that the input data is assumed to have zero background.  If that is not the case, please see the documentation for the `photutils.background` subpackage for tools to help subtract the background.

<div class="alert alert-warning alert-block"> 
    <h3>Learn More:</h3>

See the [local background subtraction notebook](photutils_local_backgrounds.ipynb) for examples of local background subtraction.
</div>


The background was already subtracted for our XDF example data.

In [ ]:
from photutils import aperture_photometry

In [ ]:
phot = aperture_photometry(data, aperture)
phot

The output is an Astropy `QTable` (Quantity Table) with sum of data values within the aperture (using the defined pixel overlap method).

The table also contains metadata, which is accessed by the `meta` attribute of the table.  The metadata is stored as a python (ordered) dictionary:

In [ ]:
phot.meta

In [ ]:
phot.meta['version']

Aperture photometry using the **'center'** method gives a slightly different (and less accurate) answer:

In [ ]:
phot = aperture_photometry(data, aperture, method='center')
phot

Now perform aperture photometry using the **'subpixel'** method with `subpixels=5`:

These parameters are equivalent to SExtractor aperture photometry.

In [ ]:
phot = aperture_photometry(data, aperture, method='subpixel', subpixels=5)
phot

## Photometric Errors

We can also input an error array to get the photometric errors.

In [ ]:
phot = aperture_photometry(data, aperture, error=error)
phot

The error array in our XDF FITS file represents only the background error.  If we want to include the Poisson error of the source we need to calculate the **total** error:

$\sigma_{\mathrm{tot}} = \sqrt{\sigma_{\mathrm{b}}^2 +
                  \frac{I}{g}}$
                  
where $\sigma_{\mathrm{b}}$ is the background-only error,
$I$ are the data values, and $g$ is the "effective gain".

The "effective gain" is the value (or an array if it's variable across an image) needed to convert the data image to count units (e.g. electrons or photons), where Poisson statistics apply.

Photutils provides a `calc_total_error()` function to perform this calculation.

In [ ]:
from photutils.utils import calc_total_error

In [ ]:
# this time include the Poisson error of the source

# our data array is in units of e-/s
# so the "effective gain" should be the exposure time
eff_gain = hdr['TEXPTIME']
tot_error = calc_total_error(data, error, eff_gain)

phot = aperture_photometry(data, aperture, error=tot_error)
phot

The total error increased only slightly because this is a small faint source.

## Units

We can also input the data (and error) units via the `unit` keyword.

In [ ]:
# input the data units
import astropy.units as u

In [ ]:
unit = u.electron / u.s
phot = aperture_photometry(data, aperture, error=tot_error, unit=unit)
phot

In [ ]:
phot['aperture_sum']

Instead of inputting units via the units keyword, `Quantity` inputs for data and error are also allowed.

In [ ]:
phot = aperture_photometry(data * unit, aperture, error=tot_error * u.adu)
phot

The `unit` will not override the data or error unit.

In [ ]:
phot = aperture_photometry(data * unit, aperture, error=tot_error * u.adu, unit=u.photon)
phot

## Performing aperture photometry at multiple positions

Now let's perform aperture photometry for three sources (all with the same aperture size).  We simply define three (x, y) positions.

In [ ]:
positions = [(90.73, 59.43), (73.63, 139.41), (43.62, 61.63)]
radius = 5.
apertures = CircularAperture(positions, r=radius)

Let's plot these three apertures on the data.

In [ ]:
plt.imshow(data, norm=norm)
apertures.plot(color='red', lw=2)

Now let's perform aperture photometry.

In [ ]:
phot = aperture_photometry(data, apertures, error=tot_error, unit=unit)
phot

Each source is a row in the table and is given a unique **id** (the first column).

## Adding columns to the photometry table

We can add columns to the photometry table.  Let's calculate the signal-to-noise (SNR) ratio of our sources and add it as a new column to the table.

In [ ]:
snr = phot['aperture_sum'] / phot['aperture_sum_err']   # units will cancel

phot['snr'] = snr
phot

Now calculate the F160W AB magnitude and add it to the table.

In [ ]:
f160w_zpt = 25.9463

# NOTE that the log10() function can be applied only to dimensionless quantities
# so we use the value() method to get the number value of the aperture sum
abmag = -2.5 * np.log10(phot['aperture_sum'].value) + f160w_zpt

phot['abmag'] = abmag
phot

Now, using the WCS defined above, calculate the sky coordinates for these objects and add it to the table.

In [ ]:
from astropy.wcs.utils import pixel_to_skycoord

In [ ]:
# convert pixel positions to sky coordinates
x, y = np.transpose(positions)
coord = pixel_to_skycoord(x, y, wcs)

# we can add the astropy SkyCoord object directly to the table
phot['sky coord'] = coord
phot

We can also add separate RA and Dec columns, if preferred.

In [ ]:
phot['ra_icrs'] = coord.icrs.ra
phot['dec_icrs'] = coord.icrs.dec
phot

If we write the table to an ASCII file using the ECSV format we can read it back in preserving all of the units, metadata, and SkyCoord objects.

In [ ]:
phot.write('my_photometry.txt', format='ascii.ecsv')

In [ ]:
# view the table on disk
!cat my_photometry.txt

Now read the table in ECSV format.

In [ ]:
from astropy.table import QTable

In [ ]:
tbl = QTable.read('my_photometry.txt', format='ascii.ecsv')
tbl

In [ ]:
tbl.meta

In [ ]:
tbl['aperture_sum']   # Quantity array

In [ ]:
tbl['sky coord']    # SkyCoord array

## Aperture photometry using Sky apertures

First, let's define the sky coordinates by converting our pixel coordinates.

In [ ]:
positions = [(90.73, 59.43), (73.63, 139.41), (43.62, 61.63)]
x, y = np.transpose(positions)
coord = pixel_to_skycoord(x, y, wcs)
coord

Now define circular apertures in sky coordinates.

For sky apertures, the aperture radius must be a `Quantity`, in either pixel or angular units.

In [ ]:
from photutils import SkyCircularAperture

In [ ]:
radius = 5. * u.pix
sky_apers = SkyCircularAperture(coord, r=radius)
sky_apers.r

In [ ]:
radius = 0.5 * u.arcsec
sky_apers = SkyCircularAperture(coord, r=radius)
sky_apers.r

When using a sky aperture in angular units, `aperture_photometry` needs the WCS transformation, which can be provided in two ways.

In [ ]:
# via the wcs keyword
phot = aperture_photometry(data, sky_apers, wcs=wcs)
phot

In [ ]:
# or via a FITS hdu (i.e. header and data) as the input "data"
phot = aperture_photometry(sci_hdulist[0], sky_apers)
phot

<div class="alert alert-warning alert-block"> 
    <h3>Learn More:</h3>
    
Aperture Photometry in the [Extended notebook](photutils_extended.ipynb):

- Bad pixel masking
- Encircled flux
- Aperture photometry at multiple positions using multiple apertures

</div>

---
# Part 2:  Image Segmentation

Image segmentation is the process where sources are identified and labeled in an image.

The sources are detected by using a S/N threshold level and defining the minimum number of pixels required within a source.

First, let's define a threshold image at 2$\sigma$ (per pixel) above the background.

In [ ]:
bkg = 0.  # background level in this image
nsigma = 2.
threshold = bkg + (nsigma * error)  # this should be background-only error

Now let's detect "8-connected" sources of minimum size 5 pixels where each pixel is 2$\sigma$ above the background.

"8-connected" pixels touch along their edges or corners. "4-connected" pixels touch along their edges. For reference, SExtractor uses "8-connected" pixels.

The result is a segmentation image (`SegmentationImage` object).  The segmentation image is the isophotal footprint of each source above the threshold: an array in which each object is labeled with an integer. As a simple example, a segmentation map containing two distinct sources might look like this:

```
0 0 0 0 0 0 0 0 0 0
0 1 1 0 0 0 0 0 0 0
1 1 1 1 1 0 0 0 2 0
1 1 1 1 0 0 0 2 2 2
1 1 1 0 0 0 2 2 2 2
1 1 1 1 0 0 0 2 2 0
1 1 0 0 0 0 2 2 0 0
0 1 0 0 0 0 2 0 0 0
0 0 0 0 0 0 0 0 0 0
```
where all of the pixels labeled `1` belong to the first source, all those labeled `2` belong to the second, and all null pixels are designated to be background.

In [ ]:
from photutils import detect_sources

In [ ]:
npixels = 5
segm = detect_sources(data, threshold, npixels)

print('Found {0} sources'.format(segm.nlabels))

Display the segmentation image.

In [ ]:
from photutils.utils import random_cmap

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 8))
ax1.imshow(data, norm=norm)
lbl1 = ax1.set_title('Data')
ax2.imshow(segm, cmap=segm.cmap())
lbl2 = ax2.set_title('Segmentation Image')

It is better to filter (smooth) the data prior to source detection.

Let's use a 5x5 Gaussian kernel with a FWHM of 2 pixels.

In [ ]:
from astropy.convolution import Gaussian2DKernel
from astropy.stats import gaussian_fwhm_to_sigma

In [ ]:
sigma = 2.0 * gaussian_fwhm_to_sigma    # FWHM = 2 pixels
kernel = Gaussian2DKernel(sigma, x_size=5, y_size=5)
kernel.normalize()

ssegm = detect_sources(data, threshold, npixels, filter_kernel=kernel)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 8))
ax1.imshow(segm, cmap=segm.cmap())
lbl1 = ax1.set_title('Original Data')
ax2.imshow(ssegm, cmap=ssegm.cmap())
lbl2 = ax2.set_title('Smoothed Data')

### Source deblending

Note above that some of our detected sources were blended.  We can deblend them using the `deblend_sources()` function, which uses a combination of multi-thresholding and watershed segmentation.

How the sources are deblended can be controlled with the two keywords `nlevels` and `contrast`:
- `nlevels` is the number of multi-thresholding levels to use
- `contrast` is the fraction of the total source flux that a local peak must have to be considered as a separate object

In [ ]:
from photutils import deblend_sources

In [ ]:
segm2 = deblend_sources(data, ssegm, npixels, filter_kernel=kernel,
                        contrast=0.001, nlevels=32)

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 8))
ax1.imshow(data, norm=norm)
ax1.set_title('Data')
ax2.imshow(ssegm, cmap=ssegm.cmap())
ax2.set_title('Original Segmentation Image')
ax3.imshow(segm2, cmap=segm2.cmap())
ax3.set_title('Deblended Segmentation Image')

print('Found {0} sources'.format(segm2.max))

## Measure the photometry and morphological properties of detected sources

In [ ]:
from photutils import source_properties

In [ ]:
catalog = source_properties(data, segm2, error=error, wcs=wcs)

`catalog` is a `SourceCatalog` object.  It behaves like a list of `SourceProperties` objects, one for each source.

In [ ]:
catalog

In [ ]:
catalog[0]   # the first source

In [ ]:
catalog[0].xcentroid    # the xcentroid of the first source

Please go [here](http://photutils.readthedocs.org/en/latest/api/photutils.segmentation.SourceProperties.html#photutils.segmentation.SourceProperties) to see the complete list of available source properties.

We can create a Table of isophotal photometry and morphological properties using the ``to_table()`` method of `SourceCatalog`:

In [ ]:
tbl = catalog.to_table()
tbl

Additional properties (not stored in the table) can be accessed directly via the `SourceCatalog` object.

In [ ]:
# get a single object (id=12)
obj = catalog[11]
obj.id

In [ ]:
obj

Let's plot the cutouts of the data and error images for this source.

In [ ]:
fig, ax = plt.subplots(figsize=(12, 8), ncols=3)
ax[0].imshow(obj.make_cutout(segm2.data))
ax[0].set_title('Source id={} Segment'.format(obj.id))
ax[1].imshow(obj.data_cutout_ma)
ax[1].set_title('Source id={} Data'.format(obj.id))
ax[2].imshow(obj.error_cutout_ma)
ax[2].set_title('Source id={} Error'.format(obj.id))

<div class="alert alert-warning alert-block"> 
    <h3>Learn More:</h3>
    
Image Segmentation in the [Extended notebook](photutils_extended.ipynb):

- Define a subset of source labels
- Define a subset of source properties
- Additional sources properties, such a cutout images
- Define the approximate isophotal ellipses for each source

</div>

<div class="alert alert-warning alert-block"> 
    <h3>PSF Photometry:</h3>
    
See the two additional notebooks on using `photutils` for PSF-fitting photometry:

- [Gaussian PSF Photometry](gaussian_psf_photometry.ipynb)
- [Image-based PSF Photometry](image_psf_photometry_withNIRCam.ipynb)

</div>